# Семинар №12
## Pandas
  
  
Сегодня в меню:
  * 1. Первое знакомство
  * 2. Глобальные настройки
  * 3. Типы данных в pandas
      * 3.1 pandas.Series
      * 3.2 pandas.DataFrame
  * 4. Базовые методы pandas.Series
  * 5. Базовые методы pandas.DataFrame
  * 6. Работа со строками и столбцами
      * 6.1. Доступ к элементам
      * 6.2. Добавление, удаление, переименование колонок и строк
      * 6.3. Итерирование
      * 6.4. Сортировка
  * 7. Статистические функции
  * 8. Применение пользовательских функций
  * 9. Работа со строковыми колонками
  * 10. Агрегации, трансформации и фильтрации
      * 10.1. Скользящие окна
      * 10.2. Работа с группами
      * 10.3 Сводные таблицы
  * 11. Пропуски и ошибки в данных
  * 12. Слияния и конкатенации таблиц
  * 13. Чтение таблиц
  * 14. Визуализация
<img src='https://miro.medium.com/max/1838/1*6d5dw6dPhy4vBp2vRW6uzw.png' width=600>

В этом ноутбуке мы познакомимся с основными фичами pandas. Эта библиотека огромна, поэтому мы сможем лишь немного поиграться с доступной функциональностью. Для детального ознакомления стоит почитать документацию

### 1. Первое знакомство

In [ ]:
import pandas as pd  # Стандартный алиас для pandas

In [ ]:
# Читаем датасет из csv файла
coins = pd.read_csv('coins.csv')  

In [ ]:
# Посмотрим что получилось
coins

Поясним значения хранящиеся в колонках
 - date - дата измерений
 - name - полное название монеты
 - symbol - сокращенное название монеты
 - price - средняя цена монеты за торговый день в USD
 - txCount - количество транзакций в сети данной монеты
 - txVolume - объем монет переведенных между адресами в сети данной монеты
 - activeAddresses - количество адресов совершавших а данный день транзации в сети данной монеты
 - open - цена монеты в начале торгов данного дня
 - close - цена монеты в конце торгов данного дня
 - high - самая высокая цена данной монеты в течение данного торгового дня
 - low - самая низкая цена данной монеты в течение данного торгового дня
 - volume - объем торгов данной монетой на биржах в данный день
 - market - капитализация данной монеты в данный день

По выводу можно получить общее представление о таблице. Видно, что в таблице могут храниться как числовые так и категориальные (строковые) данные, количество строк и столбцов. Всю эту информацию и многое другое можно получить и в коде.

Для удобства мы далее ограничим размер вывода pandas в jupyter notebook.

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
coins.shape  # Размер таблицы

In [ ]:
coins.head(3)  #  Первые 3 строки таблицы. Аналогично tail для последних строк

In [ ]:
coins.describe()  # Простейшие статистики для числовых колонок

In [ ]:
coins['price']  # Можно выбрать отдельную колонку

In [ ]:
coins['high'] - coins['low']  # Над колонками работают поэлементные операции

In [ ]:
coins['spread'] = coins['high'] - coins['low']  # Можно добавлять новые колонки в таблицу

In [ ]:
del coins['spread']  # Удаляем добавленную колонку. Можно иначе: coins.pop('spread')

Колонки в таблицах pandas ведут себя как массивы неизменяемой длины. Приятное отличие состоит в том, что колонки имеют много полезных для аналитики методов. Например,

In [ ]:
coins['symbol'].value_counts() # Количества записей в таблице для каждой монеты

In [ ]:
# Самая высокая цена монеты во всей таблице. 
coins['price'].max()

In [ ]:
# Для колонок работают стандартные методы индексации списков, но есть и другие...
coins['txVolume'][:4]

Для колонок определены бинарные операции и методы которые возвращают специальную колонку из булевских значений. Истина будет храниться только для тех индексов для которых выполнено условие. Лучше объяснит пример

In [ ]:
# Строки колонки для которых txCount был больше 1000
coins['txCount'] > 1000

In [ ]:
# Строки в которых значение попадает в заранее заданный набор
coins['symbol'].isin(['BTC', 'ADA'])

In [ ]:
# Строки колонки в которых имя монеты содержит букву 'C' 
coins['symbol'].str.contains('C')

In [ ]:
# Для колонок можно составлять сложные логические выражения (скобки обязательны!)
((coins['price'] > 18000) | (coins['market'] > 200000)) & (coins['symbol'] == 'BTC')

Основное назначение получающихся булевых колонок в том, что они могут быть использованы как индексирующий объект. Эдакий продвинутый slicing для колонок и таблиц

In [ ]:
# Выбрать из таблицы записи когда цена биткоина была меньше 100 USD
coins[(coins['price'] < 100) & (coins['symbol'] == 'BTC')]

In [ ]:
# Количество транзакций лайткойна
coins['txCount'][coins['symbol'] == 'LTC']

**Задание**: У какой монеты была самая высокая цена?

In [ ]:
# your code here

Для таблиц есть индексирование и по строкам и по колонкам.

In [ ]:
# Выбрать из таблицы максимальную цену и объем торгов за дни когда цена дож была выше 1 цента
coins.loc[
    (coins['symbol'] == 'DOGE') & (coins['price'] > 0.01),
    ['high', 'volume']
]

Начиная с этого момента вы уже можете делать простейшие выборки из данных и немного их преобразовывать. Конечно, не забывайте  использовать help() и tab-completion чтобы узнать больше о возможностях pandas. В голове стоит держать такое правило: если вам очень хочется пройтись циклом по таблице и что-то сделать с данными, очень вероятно, что вы делаете что-то неправильно и, скорее всего, в pandas есть инструмент для вашей задачи. Оставшаяся часть ноутбука посвящена перечислению фич, и может показаться скучной, но не зная этих фич вы каждый раз будете переизобретать велосипед.

### 2. Глобальные настройки

Возможно вам захочется сконфигурировать работу pandas. Большинство глобальных настроек касаются отображения таблицы или данных

In [ ]:
# Узнать максимальное кол-во отображаемых в таблице с
pd.get_option('display.max_columns')

In [ ]:
# Узнать больше про опцию. Список всех опций тут: 
# https://pandas.pydata.org/pandas-docs/version/0.23.4/options.html#available-options
pd.describe_option('display.max_columns')

In [ ]:
# Назначть максимальное число отображаемых строк
pd.set_option('display.max_columns', 15)

### 3. Типы данных в pandas

В pandas есть два основных типа данных. pandas.Series и pandas.DataFrame.

#### 3.1 pandas.Series

Про pandas.Series стоит думать как про одномерный массив фиксированного размера из данных одного типа. В отличие от массивов доступ к элементам может быть по нечисловому индексу. Индекс стоит понимать как имена строк.

Объекты типа pandas.Series очень часто используются как возвращаемые значения в pandas.

In [ ]:
# pandas.Series хранящий буквы со стандартной индексацией
pd.Series(data=['BTC', 'LTC', 'DOGE', 'DASH'])

In [ ]:
# pandas.Series хранящий числа типа str cо специальным индексом
pd.Series(data=['BTC', 'ETH', 'XRP'], 
          index=['first_coin', 'second_coin', 'third_coin'], dtype=str)

In [ ]:
# pandas.Series созданный из словаря
pd.Series({'first_coin': 'BTC', 'second_coin': 'ETH', 'third_coin': 'XRP'})

In [ ]:
# Стандартная индексация
sequence = pd.Series(data=['!', 'the', 'Moon', 'To'])

In [ ]:
sequence[[3, 1, 2, 0, 0, 0]]  # Доступ к нескольким элементам по их позициям 

In [ ]:
sequence[3]  # Доступ к элементу по позиции

In [ ]:
# Спец индексация
sequence = pd.Series(data=['!', 'the', 'Moon', 'To'], 
                     index=['04-11-2018', '02-11-2018', '03-11-2018', '01-11-2018'])

In [ ]:
sequence[['01-11-2018', '02-11-2018', '03-11-2018', '04-11-2018']]  # Доступ к нескольким элементам по их индексам

In [ ]:
sequence['01-11-2018']  # Доступ к элементу по индексу

In [ ]:
# Чтобы не засорять global namespace
del sequence

#### 3.2 pandas.DataFrame

Данные типа pandas.DataFrame это двумерный массив (переменного размера) разнородных данных (но однородных по колонкам). Лучше всего предстаставлять себе pandas.DataFrame как набор колонок, где каждая колонка это pandas.Series.

In [ ]:
# Создание простейшей таблицы
pd.DataFrame(data=['BTC', 'LTC', 'XRP'])

In [ ]:
# Cоздание таблицы с именами колонок
pd.DataFrame(data=[['BTC', 10000],['LTC', 200],['XRP', 1]], 
             columns=['symbol', 'price'])

In [ ]:
# Cоздание таблицы с именами колонок и указанием индекса
pd.DataFrame(
    data=[['BTC', 10000],['LTC', 200],['XRP', 1]],
    columns=['symbol', 'price'],
    # DatetimeIndex - это специальный тип данных для индексов содержащих время
    index=pd.DatetimeIndex(['01-11-2018', '03-11-2018', '23-08-2018'])
)

In [ ]:
# Создание по готовым pandas.Series
pd.DataFrame({
    'symbol': pd.Series(data=['BTC','LTC'], index=pd.DatetimeIndex(['01-11-2018', '03-11-2018'])),
    'price': pd.Series(data=[10000, 200], index=pd.DatetimeIndex(['01-11-2018', '03-11-2018']))
})

Некоторые функции в pandas одинаково применимы как к строкам так и к столбцам. В этом случае, в функцию можно передать аргумент axis. Значение 0 идет по умолчанию и ему соответствует работа по строкам, 1 - по колонкам.

### 4. Базовые методы pandas.Series

Посмотрим на самые базовые методы класса pandas.Series

In [ ]:
sequence = pd.Series(data=['BTC', 'LTC', 'DOGE', 'DASH'], index=['leader', 'alternative', 'joke', 'bouncer'])

In [ ]:
# Индекс наших данных
sequence.axes

In [ ]:
# Тип наших данных
sequence.dtype

In [ ]:
# Проверка на пустоту
sequence.empty

In [ ]:
# Количество размерностей. Для pandas.Series всегда 1
sequence.ndim

In [ ]:
# Количество элементов
sequence.size

In [ ]:
# Размеры
sequence.shape

In [ ]:
# Получить данные в виде numpy массива. Низкоуровневое представление с точки зрения pandas
sequence.values

In [ ]:
# Получить первые 3 элемента
sequence.head(3)

In [ ]:
# Получить последние 2 элемента
sequence.tail(2)

In [ ]:
del sequence

### 5. Базовые методы pandas.DataFrame

Класс DataFrame огромен. Мы рассмотрим только самые часто используемые в нем методы

In [ ]:
table = pd.DataFrame(
    data=[['BTC', 10000],['LTC', 200],['XRP', 1]],
    columns=['symbol', 'price'],
    # DatetimeIndex - это специальный тип данных для индексов содержащих время
    index=pd.DatetimeIndex(['01-11-2018', '03-11-2018', '23-08-2018'])
)
table

In [ ]:
# Транспонировать таблицу
table.T

In [ ]:
# Получить индексы срок и столбцов
table.axes

In [ ]:
# Получить типы данных в колонках
table.dtypes

In [ ]:
# Проверка на пустоту
table.empty

In [ ]:
# Размеры
table.shape

In [ ]:
# Количество элементов
table.size

In [ ]:
# Низкоуровневое представление таблицы
table.values

In [ ]:
# Получить первые 3 строки
table.head(3)

In [ ]:
# Получить последние 2 строки
table.tail(2)

In [ ]:
del table

### 6. Работа со строками и столбцами

Обращаться к строкам таблицы можно по номеру или по индексу

In [ ]:
table = pd.DataFrame(
    data=[['BTC', 10000],['LTC', 200],['XRP', 1], ['BTC', 9000]],
    columns=['symbol', 'price'],
    # DatetimeIndex - это специальный тип данных для индексов содержащих время
    index=pd.DatetimeIndex(['01-11-2018', '03-11-2018', '23-08-2018', '02-11-2018'])
)
table

#### 6.1. Доступ к элементам

In [ ]:
# Обращение по номеру
table.iloc[2]

In [ ]:
# Обращение по диапазону номеров
table[1:3]

In [ ]:
# Обращение по индексу
table.loc[pd.Timestamp('01-11-2018')]

In [ ]:
# Обращение по диапазону индексов
table.loc['01-11-2018':'03-11-2018']

#### 6.2. Добавление, удаление, переименование колонок и строк

In [ ]:
# Удаление строк по их индексам. Можно использовать для удаления колонок, указав axis=1
# Посмотрите еще методы drop_duplicates and dropna
table.drop([pd.Timestamp('2018-08-23'), pd.Timestamp('2018-01-11')])

In [ ]:
# Чтобы добавить новую колонку достаточно присвоить ей значение
table['volume'] = [1e9, 1e6, 1e4, 1e9]
table

In [ ]:
# Удалять колонки очень просто
del table['volume']

In [ ]:
# Переименовывание колонок
table.rename(columns={'symbol': 'Symbol', 'price': 'Price'})

#### 6.3. Итерирование

Итерироваться по таблице можно разными способами, но обычно так не стоит делать и лучше написать решение в виде манипуляций над столбцами/колонками 

In [ ]:
# По колонкам
for column in table:
    print(column)

In [ ]:
# По колонкам как по словарю. Значения в "словаре" это pandas.Series
for column, series in table.iteritems():
    print(column)
    print(series)

In [ ]:
# По строкам как по словарю. Значения в "словаре" это pandas.Series
for index, row in table.iterrows():
    print(index)
    print(row)

#### 6.4. Сортировка

Строки и столбцы таблицы можно сортировать. Результат сортирующих методов возвращает новую таблицу!

In [ ]:
# Отсортировать строки по индексу по убыванию
table.sort_index(ascending=False)

In [ ]:
# Отсортировать колонки по возрастанию
table.sort_index(axis=1, ascending=True)

In [ ]:
# Отсортировать таблицу алгоритмом quicksort по названию, а если значения равны то по цене
table.sort_values(by=['symbol', 'price'], kind='quicksort')

In [ ]:
del table, index, row, column, series

### 7. Статистические функции

Посмотрим как pandas позволяет вычислять простейшие статистики для датасетов. Методы которые мы перечислим ниже имеют аргумет axis, который указывает вдоль какой оси надо вычислять статистику. Обычно, по-умолчанию axis=0, что соответсвует вычислениям по строкам. Если в данных есть пропуски, то они не учитываются при вычислении статистик!

In [ ]:
earnings = pd.DataFrame(
    data=[[7629.39, -9357.49, -1661.3, 8597.23],
          [560.68, None, 10.46, 3578.5],
          [487.38, 7560.38, 1090.87, -5164.93],
          [560.68, 7560.38, None, -5164.93]],
    columns=['BTC', 'DOGE', 'ADA', 'ETH'],
    index=['yesterday', 'today', 'tomorrow','aftertomorrow']
)


In [ ]:
earnings

In [ ]:
# Количество не None записей для каждой монеты
earnings.count()

In [ ]:
# Заработки по дням
earnings.sum(axis=1)

In [ ]:
# Средний заработок по каждой монете
earnings.mean()

In [ ]:
# Медианный заработок по каждой монете
earnings.median()

In [ ]:
# Моды заработка по каждой монете
earnings.mode()

In [ ]:
# "Разброс" заработка по каждой монете
earnings.std()

In [ ]:
# 25, 50 и 75 квантиль заработка по каждой монете
earnings.quantile(q=(0.25, 0.5, 0.75))

In [ ]:
# Минимальный заработок по каждой монете
earnings.min()

In [ ]:
# Максимальный заработок по каждой монете
earnings.max()

In [ ]:
# Абсолюты заработков по каждой монете
earnings.abs()

In [ ]:
# Произвдение заработков по каждой монете. Да я знаю, что это бессмысленно
earnings.prod()

In [ ]:
# Кумулятивные произведения заработков по каждой монете
earnings.cumprod()

In [ ]:
# Кумулятивные суммы заработков по каждой монете
earnings.cumsum()

In [ ]:
# Процентное изменение заработка по сравнению с предыдущим значением
earnings.pct_change()

In [ ]:
# Ковариация между заработком по BTC и по DOGE
earnings['BTC'].cov(earnings['DOGE'])

In [ ]:
# Корреляция между заработком по BTC и по DOGE
earnings['BTC'].corr(earnings['DOGE'])

Иногда хочется посмотреть все статистики сразу. Для этого есть метод describe

In [ ]:
earnings.describe()

In [ ]:
del earnings

### 8. Применение пользовательских функций

Иногда хочется применить к датасету или одной его колонке функцию. Применять можно ко всей таблице, поэлементно, поколоночно или построчно

In [ ]:
earnings = pd.DataFrame(
    data=[[7629.39, -9357.49, -1661.3, 8597.23],
          [560.68, None, 10.46, 3578.5],
          [487.38, 7560.38, 1090.87, -5164.93]],
    columns=['BTC', 'DOGE', 'ADA', 'ETH'],
    index=['yesterday', 'today', 'tomorrow']
)
earnings

Для  применения построчно/поколоночно используйте функцию apply. Если хочется применить построчно добавьте axis=1

In [ ]:
#  Посчитать дисперсию заработка для каждой монеты. В лямбду приходит pandas.Series объект каждой колонки
earnings.apply(lambda money: money.std() ** 2)

Для поэлементного применения используйте функцию applymap

In [ ]:
# Сделаем вид, что потери превратились в профит
earnings.applymap(abs)

In [ ]:
# Применять функцию можно и к отдельной колонке. В лямбду приходит отдельное значение колонки
earnings['BTC+100500'] = earnings['BTC'].apply(lambda money: money + 100500)
earnings

In [ ]:
# Если нужно взаимодействие между колонками - применяйте apply row-wise (по строкам) 
earnings['BTC+DOGE'] = earnings.apply(lambda x: x['BTC'] + x.DOGE, axis = 1)
earnings

#### Не забудьте самостоятельно посмотреть метод pipe

In [ ]:
help(earnings.pipe)

In [ ]:
del earnings

### 9. Работа со строковыми колонками

На данный момент для работы со строковыми колонками нам пришлось бы постоянно использовать метод apply. Это неудобно и поэтому в pandas сделали удобный инструмент для манипуляций со строками

In [ ]:
taxonomy = pd.DataFrame(
    data=[['BTC', 'Bitcoin'],['LTC', 'Litecoin'], ['ETC', 'Etherium'], ['DOGE', 'Doge coin']],
    columns=['symbol', 'name']
)
taxonomy

Чтобы применять строковые функции к колонке надо обратиться к свойству .str.

In [ ]:
# Привести все к нижнему регистру
taxonomy['symbol'].str.lower()

In [ ]:
# Привести все к верхнему регистру
taxonomy['name'].str.upper()

In [ ]:
# Длины строк
taxonomy['symbol'].str.len()

In [ ]:
# Убрать по краям пробельные символы
taxonomy['name'].str.strip()

In [ ]:
# Разбить строки по заданному символу
taxonomy['name'].str.split(' ')

In [ ]:
# Склеить все строки в одну. Аналог str.join
taxonomy['symbol'].str.cat(sep=' $$$ ')

In [ ]:
# Найти везде подстроку
taxonomy['name'].str.contains('coin')

In [ ]:
# Везде заменить строку на другую
taxonomy['name'].str.replace('coin', 'dough')

In [ ]:
# Повторить каждую строку указанное число раз
taxonomy['name'].str.repeat(2)

In [ ]:
# Посчитать все появления указанной строки
taxonomy['name'].str.count('coin')

In [ ]:
# Проверка наличия префикса
taxonomy['symbol'].str.startswith('B')

In [ ]:
# Проверка наличия суффикса
taxonomy['symbol'].str.endswith('C')

In [ ]:
# Найти место в строках где начинается искомая строка. 
taxonomy['name'].str.find('coin')

In [ ]:
# Проверка верхнего регистра
taxonomy['symbol'].str.isupper()

In [ ]:
# Проверка нижнего регистра
taxonomy['symbol'].str.islower()

In [ ]:
# Проверка что все символы - цифры
taxonomy['name'].str.isnumeric()

In [ ]:
#Посмотреть все возможные проверки
help(taxonomy['name'].str.isnumeric)

### 10. Агрегации, трансформации и фильтрации

#### 10.1. Скользящие окна

Мы начнем со специального случая аггрегаций данных - оконных функций. Мы поговорим о методах rolling и emw. Самостоятельно посмотрите что делает expanding.

In [ ]:
price = pd.DataFrame(
    data=[
        [0.547, 0.745],
        [0.5, 0.964],
        [0.77, 0.718],
        [0.135, 0.845],
        [0.53, None],
        [0.15, 0.795]],
    columns=['ADA', 'DOGE'],
    # Создаем специальный индекс из 6 дней начиная с 2018-11-01 
    index=pd.date_range('2018-11-01', periods=6)
)
price

Если вы хотите посчитать какую-нибудь статистическую функци (sum, mean, median, std) в скользящем окне. То есть методы rolling (обычное окно) и ewm (окно с экспоненциальным сглаживанием). Они возвращают специальные объекты которые могут аггрегировать используя стандартную или пользователскую функцию.

In [ ]:
# Найти скользящее стандартное отклонение
price.rolling(window=2).mean()

In [ ]:
# Сделать агрегацию c пользовательской функцией
price.rolling(window=2).agg(lambda series: sum(series ** 2))

In [ ]:
# Сделать агрегацию по имени функции
price.rolling(window=2).agg('max')

In [ ]:
# Сделать несколько агрегаций
price.rolling(window=2).agg(['mean', sum, lambda series: max(series ** 2)])

In [ ]:
# Сделать несколько агрегаций (для каждой колонки свои)
price.rolling(window=2).agg({'ADA': [sum, max], 'DOGE': lambda series: max(series ** 2)})

Объект ewm работает аналогично rolling с тем лишь отличием, что он назначает веса. Числам $x_0,\ldots,x_t$ будут назначены веса $(1-\alpha)^t, (1-\alpha)^{t-1}, \ldots, 1$, где $\alpha$ - параметр сглаживания. Например, экспоненциальное скользящее среднее будет вычисляться по формуле 
$$
ewm_{\alpha}(x)=\frac{\sum_{i=0}^t (1-\alpha)^{t-i} x_i}{\sum_{i=0}^t(1-\alpha)^i}
$$


Если указать **adjust=False**, подсчёт будет идти рекурсивно: ```mean[i] = (1-alpha)*mean[i-1] + alpha*x[i]```

In [ ]:
# Скользящее среднее cо сглаживанием alpha
price.ewm(alpha=0.5, adjust=False).mean()

In [ ]:
del price

#### 10.2. Работа с группами

Иногда хочется разбить данные по группам и посчитать статистики / преобразовать / пофильтровать. Для этого есть метод **groupby**. Он возвращает специальный объект для работы со сгруппированными данными.

In [ ]:
operations = pd.DataFrame(
    data=[
        ['DOGE', 'buy', 5000, 0.2],
        ['BTC', 'buy', 10, 6000],
        ['BTC', 'buy', 2.5, 5900],
        ['DOGE', 'sell', 3000, 0.1],
        ['BTC', 'sell', 4, 6200],
        ['ETH', 'sell', 10, 400],
        ['BTC', 'buy', 1, 5600],
        ['ETH', 'buy', 20, 350],
        ['ETH', 'sell', 10, 300],
        ['DOGE', 'sell', 3000, 0.15]
    ],
    columns=['symbol', 'type', 'amount', 'price'],
    index=pd.date_range('2018-01-01', periods=10)
)
operations

In [ ]:
# Сгруппировать операции по типу монеты
operations.groupby('symbol')

In [ ]:
# Сгруппировать операции по типу монеты и типу операции
operations.groupby(['symbol', 'type'])

In [ ]:
# Получить индексы строк каждой группы
operations.groupby(['symbol', 'type']).groups

In [ ]:
# Посмотреть на содержимое каждой группы 
for name, group in operations.groupby(['symbol', 'type']):
    print(name)
    print(group)

In [ ]:
# Получить группу
operations.groupby(['symbol', 'type']).get_group(('BTC', 'buy'))

Теперь посмотрим как агрегировать данные для каждой группы

In [ ]:
# Найти обороты для каждой монеты и операции
grouper = operations.groupby(['symbol', 'type'])
grouper['amount'].agg(sum)

# Есть ещё полезный метод operations.groupby(...).transform(func) - предлагается разобрать самостоятельно

In [ ]:
# Найти обороты и самые крупные сделки для каждой монеты и операции
grouper = operations.groupby(['symbol', 'type'])
grouper['amount'].agg([sum, max])

In [ ]:
# Найти обороты и среднюю цену для каждой монеты и операции
grouper = operations.groupby(['symbol', 'type'])
grouper.agg({'amount': sum, 'price': lambda series: series.mean()})

In [ ]:
# Получить размер группы в которой сидит запись
operations.groupby(['symbol', 'type']).apply(lambda series: series.shape[0])

In [ ]:
#Вывести группы по убыванию числа уникальных значений в колонке amount
operations.groupby(['symbol', 'type'])['amount'].nunique().nlargest(10)

Сгруппированные данные можно фильтровать. Вы можете решить какие группы оставить с помощью вашей функции. Она на вход принимает pandas.DataFrame и возвращает bool

In [ ]:
# Выбрать монеты и операции для которых оборот был больше 20 монет
operations.groupby(['symbol', 'type']).filter(lambda table: table['amount'].sum() >= 20)

In [ ]:
del operations, name, group

#### 10.3 Сводные таблицы

Большинство таблиц в сыром виде похожи на логи где каждая строка это запись. Никакой оргинизации в данных в этих таблицах нет, а хотелось бы. Для этого в pandas существуют сводные таблицы. 

In [ ]:
operations = pd.DataFrame(
    data=[
        ['DOGE', 'buy', 5000, 0.2, 'Joe Doe', 'success'],
        ['BTC', 'buy', 10, 6000, 'Elon Musk', 'error'],
        ['BTC', 'buy', 2.5, 5900, 'George Bush', 'success'],
        ['DOGE', 'sell', 3000, 0.1, 'John Romero', 'success'],
        ['BTC', 'sell', 4, 6200, 'Jack Ma', 'success'],
        ['ETH', 'sell', 10, 400, 'Satoshi Nakomoto', 'error'],
        ['BTC', 'buy', 1, 5600, 'Vladimir Vladimirovich', 'success'],
        ['ETH', 'buy', 20, 350, 'George Bush', 'success'],
        ['ETH', 'sell', 10, 300, 'Jack Ma', 'error'],
        ['DOGE', 'sell', 3000, 0.15, 'Joe Doe', 'success']
    ],
    columns=['symbol', 'type', 'amount', 'price', 'user', 'status'],
    index=pd.date_range('2018-01-01', periods=10)
)
operations

Лучше всего понять как они работают это разобрать пример. Найдем сколько суммарно  было куплено и продано каждого типа монет. В результирующей сводной таблице индексом будет название монеты, колонки - типы транзакций (т.е. покупки или продажи). В самих ячейках будет суммарное количество монет.

In [ ]:
operations.pivot_table(
    values='amount', 
    index='symbol', 
    columns='type', 
    aggfunc='sum'
)

Если не указывать явно аргумент aggfunc, то агрегация будет через усреднение (а не через суммирование как было в нашем примере). Расмотрим более сложный пример.

In [ ]:
operations.pivot_table(
    values='amount',
    index=['symbol', 'user'],  # Индекс может быть иерархическим и строиться по нескольким колонкам
    columns=['type', 'status'],  # Аналогично иерархическими могут быть и колонки
    aggfunc={'amount': ['sum', 'mean']}  # Можно вычислять сразу несколько агрегаций
)

И самый монструозный пример, где мы сделаем сводную таблиц для цен и объемов торгов.

In [ ]:
operations.pivot_table(
    values=['amount', 'price'],  # Можно находить статистику по разным числовым показателям
    index=['symbol', 'user'],
    columns=['type', 'status'],
    aggfunc={
        'amount': ['sum', 'mean'], 
        'price': 'max'}  # Тогда для каждого показателя надо указать какие агрегации вы хотите
)

Для построения сводных таблиц в pandas есть более простая функция. pandas.pivot. Она не делает никаких агрегаций, поэтому при ее использовании вы должны быть уверены, что на каждую ячейку результирующей сводной таблицы придется только одно значение показателя из исходной таблицы. В противном случае будет ошибка. Можете попробовать построить сводную таблицу
```
operations.pivot(values='amount', index='user', columns='user')
```
чтобы убедиться.

**Задание:** Самостоятельно разберитесь с функциями pandas.stack, pandas.unstack и pandas.melt. Вам поможет [этот tutorial](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)

### 11. Пропуски и ошибки в данных

В реальной жизни данные с которыми надо работать содержат пропуски или просто неверные значения. Мы обсудим как это можно исправить

In [ ]:
price = pd.DataFrame(
    data=[
        [0.547, 0.745],
        [-2, -1],
        [None, 0.718],
        [0.135, 0.845],
        [0.53, None],
        [None, -1]],
    columns=['ADA', 'DOGE'],
    # Создаем специальный индекс из 6 дней начиная с 2018-11-01 
    index=pd.date_range('2018-11-01', periods=6)
)
price

In [ ]:
# Найти места где есть пропуски методом isnull. Аналогично есть метод notnull
price.isnull()

In [ ]:
# Заполнить все пропуски дефолтным занчением
price.fillna(0)

In [ ]:
# Заполнить все пропуски занчениями с предыдущей строки
price.fillna(method='bfill')

In [ ]:
# Заполнить все пропуски занчениями с cледующей строки
price.fillna(method='ffill')

In [ ]:
#  Выкинуть строки содержащие nan'ы
price.dropna()

In [ ]:
#  Выкинуть колонки содержащие nan'ы
price.dropna(axis=1)

In [ ]:
# Подменить значения
price.replace({-1:0, -2:0})

In [ ]:
price

In [ ]:
del price

### 12. Слияния и конкатенации таблиц

Если у вас есть две таблицы, то информацию в них можно объединить двумя способами: конкатенация ("прилепить снизу новую таблицу") и объединение (найти объединение записей таблиц если у них хранятся одни и те же значения в выбранных колонках).

In [ ]:
operations_old = pd.DataFrame(
    data=[
        ['DOGE', 'buy', 5000, 0.2],
        ['BTC', 'buy', 10, 6000],
        ['BTC', 'buy', 2.5, 5900],
        ['DOGE', 'sell', 3000, 0.1],
        ['BTC', 'sell', 4, 6200],
    ],
    columns=['symbol', 'type', 'amount', 'price']
)

operations_new = pd.DataFrame(
    data=[
        ['ETH', 'sell', 10, 400],
        ['BTC', 'buy', 1, 5600],
        ['ETH', 'buy', 20, 350],
        ['ETH', 'sell', 10, 300],
        ['DOGE', 'sell', 3000, 0.15]
    ],
    columns=['symbol', 'type', 'amount', 'price']
)

In [ ]:
#  Сконкатенировать таблицы в указанном порядке
pd.concat([operations_old, operations_new])

In [ ]:
# При конкатенации таблицам можно назначать теги чтобы понимать, где какая таблица
pd.concat([operations_old, operations_new], keys=['old', 'new'])


In [ ]:
# Как видно при конкатенации индексы старых таблиц копируются. 
# Если вам не нужно это поведение, используйте ignore_index
operations = pd.concat([operations_new, operations_old], ignore_index=True)
operations

Данные можно собирать вместе и более умным способом. А именно, вы выбираете какие колонки вас интересуют. Далее если есть записи из двух таблиц у котороых в выбранных колонках значения совпадают мы их "объединяем". Это очень упрощенное объяснение. В pandas эта операция называется merge в SQL - join. Делать объединение можно разными сопособами, поэтому возникает несколько стратегий: left, right, inner и full. 

In [ ]:
transactions = pd.DataFrame(
    data=[
        ['DOGE', 'sell', 'Rockfeller'],
        ['DOGE', 'buy', 'J.P. Morgan'],
        ['BTC', 'buy', 'John Doe'],
        ['ADA', 'sell', 'Rick'],
        ['ETH', 'buy', 'Morty']
    ],
    columns=['symbol', 'type', 'user']
)
transactions

In [ ]:
# Объединение таблиц по типу монеты и операции
# По умолчанию производится inner join остаются записи в которых были полные совпадения
pd.merge(operations, transactions, on=['symbol', 'type'])

In [ ]:
# Левый join таблиц - все записи из первой таблицы остаются и к ним присоединяются записи из правой.
# Аналогично работает правый join
pd.merge(operations, transactions, on=['symbol', 'type'], how='left')

In [ ]:
# Outer join - собираем вместе записи из обеих таблиц
pd.merge(operations, transactions, on=['symbol', 'type'], how='outer')

In [ ]:
del operations_new, operations_old, operations, transactions

### 13. Чтение таблиц

Чтобы данные анализировать их надо где-то взять. Для этого в pandas есть множество функций. Все их названия начинаются с префикса 'read_'. Мы разберем функцию read_csv.

In [ ]:
# Прочитать таблицу ни о чем не задумываясь
pd.read_csv('coins.csv')

In [ ]:
# Прочитать таблицу и использовать колонку date как индекс 
pd.read_csv('coins.csv', index_col='date')

In [ ]:
# Прочитать таблицу и дать pandas подсказки по поводу типов данных в колонках symbol и name
pd.read_csv('coins.csv', dtype={'symbol': str, 'name': str})

### Запись таблиц

In [ ]:
coins = pd.read_csv('coins.csv')

coins.to_csv('file_name.csv')

### 14. Визуализация

В pandas есть очень простые средства визуализации. Они подойдут для быстрого анализа, но если хочется сделать красиво и информативно, то вы не по адресу. Любая визуализация делается через аттрибут plot у объекта pandas.Series или pandas.DataFrame.

Стоит отметить что для отображения графиков нужно использовать магическую команду для jupyter
```
%matplotlib
```
При этом график появится в отдельном окне. Там же будет несколько полезных кнопок для редактирования и сохранения графика. Если хочется чтобы графики отображались прямо в ноутбуке то надо передать аргумент inline
```
%matplotlib inline
```

In [ ]:
%matplotlib inline

In [ ]:
price = pd.DataFrame(
    data=[
        [0.547, 0.745],
        [0.5, 0.964],
        [0.77, 0.718],
        [0.135, 0.845],
        [0.53, None],
        [0.15, 0.795]],
    columns=['ADA', 'DOGE'],
    index=pd.date_range('2018-11-01', periods=6)
)
price

In [ ]:
# Нарисовать график цены монеты ADA. 
# Так как индекс в нашей таблице временный, то pandas догадывается, что его можно использовать
# как ось времени на графиках.
price['ADA'].plot()

In [ ]:
# Можно на одном графике нарисовать поведение цены сразу нескольких монет
price[['ADA', 'DOGE']].plot()

In [ ]:
# Цену можно рисовать и в виде столбцовой диаграммы c различными настройками
price.plot.bar(stacked=True)

In [ ]:
# Горизонтальная столбцовая диаграмма
price.plot.barh()

In [ ]:
# Гистограмма
price.plot.hist(bins=10)

In [ ]:
# Ящики с усами
price.plot.box()

In [ ]:
# Графики с заливкой
price.plot.area()

In [ ]:
# Если хочется нарисовать пары каких либо значений в виде точек на плоскости, то подойдет scatter plot
price.plot.scatter(x='ADA', y='DOGE')

In [ ]:
# И на десерт - пирожковая диаграмма
price.plot.pie(subplots=True, figsize=(15,7.5))

In [ ]:
del price

# NB


## Другие инструменты работы с данными

Если ваш датасет не слишком большой, то pandas это де-факто лучший инструмент в наличии. В противном случае есть альтернативы:
 - [numpy](http://www.numpy.org/) - библиотека для низкоуровневой работы с многомерными массивами из примитивных типов данных. Подходит для "векторизованной" обработки. Numpy - ваш последний шанс все еще писать на обычном питоне, без биндингов на С. Подходит для задач быстрой обработки не слишком больших данных. На самом деле внутри, pandas использует numpy.
 - [Graphlab](https://turi.com/) - Платная питоновская библиотека для работы как большими данными так и с маленькими датасетами. Данные представляются как SFrame объекты, во многом похожие на DataFrame из pandas.
 - [Hadoop](https://hadoop.apache.org/) - стек технологии для работы с очень большими данными, в частности есть классический MapReduce.
 


## Другие инструменты визуализации


Для визуализации библиотек намного больше. Вот самые популярные
 - [Pandas](https://pandas.pydata.org/) - для быстрой и очень простой визуализации. Любая мелкая настройка потребует знания matplotlib
 - [Seaborn](https://seaborn.pydata.org/) - широкий но фиксированный набор сложных визуализаций с красивыми цветовыми схемами. Библиотека является надстройкой над matplotlib, и, опять же, тонкие настройки потребуют знания matplotlib.
   - [Галерея с примерами](https://seaborn.pydata.org/examples/index.html)
   - [Руководство](https://seaborn.pydata.org/tutorial.html)
 - [ggplot](http://ggplot.yhathq.com/) - многообещающая библиотека скопированная из R. Пока на этапе развития
 - [Bokeh](https://bokeh.pydata.org/en/latest/) - библиотека для визуализации в web.
 - [Plot.ly](https://plot.ly/) - библиотека для интерактивных и статических визуализаций. Нацелена на работу в облаке, но можно работать и локально.
   - [Галерея с примерами](https://plot.ly/python/)
   - [Краткое введение](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf)